In [1]:
import aiohttp
import asyncio
import pandas as pd
import datetime as dt
from bs4 import BeautifulSoup as bs

async def get_request_results(query):
    response = await aiohttp.request('GET', query)
    return (await response.text())

async def stock_prices_nyse(stock_name, date_start, date_end, page_number=1, df=pd.DataFrame()):
    def set_header(df_header):
        '''reset the header row'''
        df_header_ = df_header.copy()
        df_header_.columns = df_header_.loc[0].tolist()
        df_header_ = df_header_.drop(labels=0)
        return df_header_

    if not df.empty:
        df_ = df.copy()
    else:
        df_ = df

    query = 'http://thestockmarketwatch.com/stock/stock-data.aspx?symbol={stock_name}&action=showHistory&page={page_number}&perPage=150&startMonth={start_month}&startDay={start_day}&startYear={start_year}&endMonth={end_month}&endDay={end_day}&endYear={end_year}'.format(
        stock_name=stock_name, start_day=date_start.timetuple()[2], start_month=date_start.timetuple()[1] - 1, start_year=date_start.timetuple()[0],
        end_day=date_end.timetuple()[2], end_month=date_end.timetuple()[1] - 1, end_year=date_end.timetuple()[0], page_number=page_number)
#     response = requests.get(query)
#     response_text = await get_request_results(query)
    response = await aiohttp.request('GET', query)
    response_text = await response.text()

    # find page numbers
    bs_page = bs(response_text, 'lxml')
    table_data = bs_page.find_all('table', {'class': 'qm_history_historyContent'})
    if table_data:
        df_temp = pd.read_html(str(table_data[0]))[0]
        df_temp = set_header(df_temp)
        df_ = df_.append(df_temp)
        span_page_number = bs_page.find_all('span', {'class': 'qm_text'})
        if len(span_page_number) > 0:
            for span in span_page_number:
                text_ = span.text
                if 'of ' in text_:
                    total_pages = int(text_.split()[-1])
                    if page_number < total_pages:
                        page_number += 1
                        return (await stock_prices_nyse(stock_name, date_start, date_end, page_number, df_))
    return df_

async def do_work(url):
    df_returned = await stock_prices_nyse(url, dt.datetime(2015, 1, 1), dt.datetime(2016, 10, 10))
    print(url, df_returned.shape, df_returned.head())

stocks = ['aapl', 'mmm', 'aac', 'moq']
coroutines = [asyncio.ensure_future(do_work(stock)) for stock in stocks]
event_loop = asyncio.get_event_loop()

event_loop.run_until_complete(asyncio.wait(coroutines))
event_loop.close()

aac (454, 11)        Date   Open    High     Low  Close  Volume    Chg   % Chg Adj. Close  \
1  10/10/16  17.32   17.68   17.29  17.37  107181   0.21   1.22%      17.37   
2  10/07/16  17.17   17.28   16.98  17.16  137096  -0.01  -0.06%      17.16   
3  10/06/16  17.17   17.30  17.045  17.17  158078  -0.07  -0.41%      17.17   
4  10/05/16  17.35   17.56  17.075  17.24  155812  -0.11  -0.63%      17.24   
5  10/04/16  17.45  17.544   16.88  17.35  215948  -0.26  -1.48%      17.35   

  Trade Val # Trades  
1     1.87m      984  
2     2.34m     1259  
3     2.72m     1184  
4     2.68m     1629  
5     3.72m     1930  
aapl (454, 11)        Date    Open    High     Low   Close    Volume       Chg  % Chg  \
1  10/10/16  115.02  116.75  114.72  116.05  36235956    1.9798  1.74%   
2  10/07/16  114.31  114.56  113.51  114.06  24358443  0.169132  0.15%   
3  10/06/16  113.70  114.34  113.13  113.89  28779313  0.835709  0.74%   
4  10/05/16  113.40  113.66  112.69  113.05  21453089  0.04974